<a href="https://colab.research.google.com/github/thuvarahan97/HomeAlone2020_TeamGenetriX/blob/master/HomeAlone2020_GenetriX_Final.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import pandas as pd   
import numpy as np    
import statsmodels.api as sm
# Import label encoder 
from sklearn import preprocessing 
# importing one hot encoder from sklearn 
from sklearn.preprocessing import OneHotEncoder 

from sklearn.compose import ColumnTransformer

from sklearn.model_selection import train_test_split

In [0]:
traindata=pd.read_csv('Train.csv',sep=",")
testdata=pd.read_csv('Test.csv',sep=",")

In [0]:
traindata['GENDER'] = traindata['GENDER'].replace(['MALE','FEMALE'], [1,-1]) 
testdata['GENDER'] = testdata['GENDER'].replace(['MALE','FEMALE'], [1,-1]) 

In [0]:
traindata['CATEGORY'] = traindata['CATEGORY'].replace(['GENERAL','SC','ST'], [-1,0,1]) 
testdata['CATEGORY'] = testdata['CATEGORY'].replace(['GENERAL','SC','ST'], [-1,0,1]) 

In [0]:
traindata.ASSETS = [x.strip('Rs ').split("\n")[0] for x in traindata.ASSETS]
testdata.ASSETS = [x.strip('Rs ').split("\n")[0] for x in testdata.ASSETS]

In [0]:
traindata.LIABILITIES = [x.strip('Rs ').split("\n")[0] for x in traindata.LIABILITIES]
testdata.LIABILITIES = [x.strip('Rs ').split("\n")[0] for x in testdata.LIABILITIES]

In [0]:
traindata['ASSETS'] = traindata['ASSETS'].replace(['Not Available'], [-1])
traindata['ASSETS'] = traindata['ASSETS'].replace(['Nil'], [-2]) 

testdata['ASSETS'] = testdata['ASSETS'].replace(['Not Available'], [-1])
testdata['ASSETS'] = testdata['ASSETS'].replace(['Nil'], [-2]) 
testdata['ASSETS'] = testdata['ASSETS'].replace(['`'], [-2]) 

In [0]:
traindata['LIABILITIES'] = traindata['LIABILITIES'].replace(['Not Available'], [-1])
traindata['LIABILITIES'] = traindata['LIABILITIES'].replace(['NIL'], [-2])

testdata['LIABILITIES'] = testdata['LIABILITIES'].replace(['Not Available'], [-1])
testdata['LIABILITIES'] = testdata['LIABILITIES'].replace(['NIL'], [-2])

In [0]:
traindata['ASSETS'] = traindata['ASSETS'].replace(',', '', regex=True).astype(int)
testdata['ASSETS'] = testdata['ASSETS'].replace(',', '', regex=True).astype(int)

In [0]:
traindata['LIABILITIES'] = traindata['LIABILITIES'].replace(',', '', regex=True).astype(int)
testdata['LIABILITIES'] = testdata['LIABILITIES'].replace(',', '', regex=True).astype(int)

In [249]:
pd.to_numeric(traindata['ASSETS'])
pd.to_numeric(testdata['ASSETS'])

0        6801126
1       30364808
2       69345867
3       28350217
4             -2
         ...    
492    209384539
493    567940114
494    139714395
495    245357182
496     25940000
Name: ASSETS, Length: 497, dtype: int64

In [250]:
pd.to_numeric(traindata['LIABILITIES'])
pd.to_numeric(testdata['LIABILITIES'])

0        5364747
1        2180647
2              0
3        3676869
4       35352263
         ...    
492    104302653
493     91408506
494     40743724
495     27807304
496      6100000
Name: LIABILITIES, Length: 497, dtype: int64

In [251]:
traindata.head()

,STATE,CONSTITUENCY,NAME,WINNER,PARTY,GENDER,CRIMINAL\nCASES,AGE,CATEGORY,EDUCATION,ASSETS,LIABILITIES,TOTAL ELECTORS
0,Lakshadweep,LAKSHADWEEP,DR. MOHAMMED SADIQUE KP,0,JD(U),1,0,64,1,Post Graduate,49783111,2887460,55189
1,Lakshadweep,LAKSHADWEEP,MOHAMMED FAIZAL PP,1,NCP,1,2,43,1,Post Graduate,938641,0,55189
2,Lakshadweep,LAKSHADWEEP,HAMDULLAH SAYEED,0,INC,1,0,36,1,Graduate Professional,95047817,445771,55189
3,Daman & Diu,DAMAN & DIU,LALUBHAI BABUBHAI PATEL,1,BJP,1,0,63,-1,8th Pass,213993750,9969514,121740
4,Jammu & Kashmir,LADAKH,SAJJAD HUSSAIN,0,IND,1,0,37,1,Post Graduate,109000,0,179232


In [0]:
traindata['BIAS']=1
testdata['BIAS']=1

In [0]:
independentVar= ['STATE',
                 'CONSTITUENCY',
                 'PARTY',
                 'GENDER',
                 'CRIMINAL\nCASES',
                 'AGE',
                 'CATEGORY',
                 'EDUCATION',
                 'ASSETS',
                 'LIABILITIES',
                 'TOTAL ELECTORS',
                  'BIAS']

In [0]:
Y_train=traindata.WINNER
X_train=traindata[independentVar]

In [0]:

X_test_all = testdata[independentVar]

In [256]:
for i in X_train.columns:
    if X_train[i].dtype=='object':
        print(i,': ',str(len(X_train[i].unique())))

STATE :  34
CONSTITUENCY :  391
PARTY :  121
CRIMINAL
CASES :  26
EDUCATION :  13


In [0]:
object_cols = [col for col in X_train.columns if X_train[col].dtype == "object"]
numeric_cols=[col for col in X_train.columns if X_train[col].dtype !='object']



In [0]:
low_card_cols = [col for col in object_cols if X_train[col].nunique() < 130]
high_card_cols = list(set(object_cols)-set(low_card_cols))


In [0]:
OH_encoder=OneHotEncoder(handle_unknown='ignore', sparse=False)
OH_cols_train = pd.DataFrame(OH_encoder.fit_transform(X_train[low_card_cols]))
OH_cols_test_all = pd.DataFrame(OH_encoder.transform(X_test_all[low_card_cols]))
OH_cols_train.index = X_train.index
OH_cols_test_all.index = X_test_all.index
X_train = X_train.drop(low_card_cols, axis=1)
X_test_all = X_test_all.drop(low_card_cols, axis=1)

X_train = pd.concat([X_train, OH_cols_train], axis=1)
X_test_all = pd.concat([X_test_all, OH_cols_test_all], axis=1)

In [0]:
X_train=X_train.drop('CONSTITUENCY',axis=1)
X_test_all=X_test_all.drop('CONSTITUENCY',axis=1)

In [261]:
X_train

,GENDER,AGE,CATEGORY,ASSETS,LIABILITIES,TOTAL ELECTORS,BIAS,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,...,154,155,156,157,158,159,160,161,162,163,164,165,166,167,168,169,170,171,172,173,174,175,176,177,178,179,180,181,182,183,184,185,186,187,188,189,190,191,192,193
0,1,64,1,49783111,2887460,55189,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
1,1,43,1,938641,0,55189,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
2,1,36,1,95047817,445771,55189,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
3,1,63,-1,213993750,9969514,121740,1,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,1,37,1,109000,0,179232,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1495,1,33,0,1205263,47000,1823664,1,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1496,1,66,0,62834200,6744375,1823664,1,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
1497,1,47,0,11021433,1500000,1823664,1,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1498,1,41,0,1082600,0,1823664,1,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0


In [0]:
X_train1,X_test,Y_train1,Y_test=train_test_split(X_train,Y_train,train_size=0.9)

In [263]:
X_train1

,GENDER,AGE,CATEGORY,ASSETS,LIABILITIES,TOTAL ELECTORS,BIAS,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,...,154,155,156,157,158,159,160,161,162,163,164,165,166,167,168,169,170,171,172,173,174,175,176,177,178,179,180,181,182,183,184,185,186,187,188,189,190,191,192,193
1499,1,49,0,18630915,4997011,1823664,1,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1014,1,52,-1,25526590,8814643,1686215,1,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
1254,1,68,-1,33436760,600000,1744762,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
305,-1,32,1,4946922,950000,1454436,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1330,1,59,-1,43292483,4230614,1768777,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
547,1,61,1,28675553,679000,1558952,1,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
1238,1,53,-1,167895109,3746607,1742883,1,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
693,1,49,-1,60045532,4039841,1612512,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
24,1,44,0,686891,0,646729,1,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0


In [0]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()

X_train = scaler.fit_transform(X_train)
X_train1 = scaler.fit_transform(X_train1)
X_test = scaler.transform(X_test)
X_test_all = scaler.transform(X_test_all)

In [0]:
# from sklearn.ensemble import RandomForestClassifier
# from sklearn.datasets import make_classification

# model = RandomForestClassifier(n_estimators=2, random_state=0)
# model1 = RandomForestClassifier(n_estimators=2, random_state=0)
import xgboost as xgb

model = xgb.XGBClassifier(learning_rate=0.1, 
                          n_estimators=100,
                          max_depth=9,
                          min_child_weight=10,
                          reg_alpha=0.01,
                          objective='binary:logistic',
                          nthread=2,
                          colsample_bytree=0.7,
                          )
model1 = xgb.XGBClassifier(learning_rate=0.1, 
                          n_estimators=100,
                          max_depth=9,
                          min_child_weight=10,
                          reg_alpha=0.01,
                          objective='binary:logistic',
                          nthread=2,
                          colsample_bytree=0.7,
                          )

In [266]:
# fit the keras model on the dataset
model.fit(X_train, Y_train)
model1.fit(X_train1, Y_train1)

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=0.7, gamma=0,
              learning_rate=0.1, max_delta_step=0, max_depth=9,
              min_child_weight=10, missing=None, n_estimators=100, n_jobs=1,
              nthread=2, objective='binary:logistic', random_state=0,
              reg_alpha=0.01, reg_lambda=1, scale_pos_weight=1, seed=None,
              silent=None, subsample=1, verbosity=1)

In [0]:
# make probability predictions with the model
y_pred_all = model.predict(X_test_all)


In [268]:
count_0 = 0
count_1 = 0

for val in y_pred_all:
  if val == 0:
    count_0 += 1
  elif val == 1:
    count_1 += 1

print (count_0, count_1)

381 116


In [0]:
# make probability predictions with the model
y_pred = model1.predict(X_test)

# # round predictions
# y_pred_rounded1 = [round(x[0]) for x in y_pred1]

In [0]:
# from sklearn.metrics import confusion_matrix
# confusion_matrix = confusion_matrix(y_test, y_pred)
# print(confusion_matrix)

In [271]:
from sklearn.metrics import classification_report
print(classification_report(Y_test, y_pred))

              precision    recall  f1-score   support

           0       0.85      0.91      0.88       117
           1       0.58      0.42      0.49        33

    accuracy                           0.81       150
   macro avg       0.72      0.67      0.69       150
weighted avg       0.79      0.81      0.79       150



In [0]:
output = pd.DataFrame(testdata['NAME'])
output['WINNER'] = y_pred_all

output_csv = output.to_csv('GenetriX.csv')